In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import mean_squared_error

In [ ]:
def gru_model(X_train, X_test, y_train, y_test):
    

    input_shape = (X_train.shape[1], X_train.shape[2])  # Shape of input data

    # Create and compile the GRU model
    model = Sequential([
        GRU(50, input_shape=input_shape),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    history = model.fit(X_train, y_train, epochs=100, batch_size=60, validation_data=(X_test, y_test), verbose=1)

    return history

In [ ]:
def evaluate_gru_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    print("Mean Squared Error: {mse}")